In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import kurtosis, skew
from scipy.stats import gaussian_kde
from math import sqrt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.models import vgg16 
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, r2_score
from collections import defaultdict
import pickle
import os
import time
import copy
import random

In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
batch_size = 64

trainset = torchvision.datasets.Flowers102(root = './assets/flowers102', split = 'train', download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers=2)

testset = torchvision.datasets.Flowers102(root = './assets/flowers102', split = 'test', download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers=2)

valset = torchvision.datasets.Flowers102(root = './assets/flowers102', split = 'val', download = True, transform = transform)
valloader = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True, num_workers=2)

In [ ]:
class CNN_FROM_SCRATCH(nn.Module):
    def __init__(self, kernel_size = 3):
        super(CNN_FROM_SCRATCH, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = kernel_size, padding = 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size = kernel_size, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0, dilation=1, ceil_mode=False)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128*64*64, 1024, bias = True),
            nn.ReLU(),
            nn.Linear(1024, 512, bias = True),
            nn.ReLU(),
            nn.Dropout(0.5, inplace = False),
            nn.Linear(512, 102, bias = True)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 128*64*64)
        x = self.classifier(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN_FROM_SCRATCH()
model.to(device)

print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

# validation function
def validate(model, val_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    for int, data in enumerate(val_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        output = model(data)
        loss = criterion(output, target)
        
        val_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss/len(val_dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(val_dataloader.dataset)
    
    return val_loss, val_accuracy

# training function
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(train_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    
    return train_loss, train_accuracy

def predict(model, data):
    model.eval()
    for i, data in enumerate(data):
        data = data[0].to(device)
        output = model(data)
        _, preds = torch.max(output.data, 1)
    return preds

In [ ]:
train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(5):
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    val_epoch_loss, val_epoch_accuracy = validate(model, valloader)
    print(f'Epoch {epoch+1} | Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_accuracy:.2f} | Validation Loss: {val_epoch_loss:.4f}, Validation Acc: {val_epoch_accuracy:.2f}')
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
end = time.time()

print((end-start)/60, 'minutes')